In [1]:
# default_exp data_manager

In [2]:
#export
from datasets import load_dataset
import numpy as np
import torch
from transformers import GPT2Tokenizer
from kirby.run_params import RunParams

# Data Manager

> Prepares and Loads data

In [12]:
#export
class DataManager():
    def __init__(self, run_params):
        self.run_params = run_params
        self.block_size = 128
    # Load, Tokenize, and Augment data
    def prepare_data(self):
        train_ds, val_ds = map(self.prepare_ds, ('train', 'valid'))
        return train_ds, val_ds
    
    def prepare_ds(self, split):
        tokenizer = GPT2Tokenizer.from_pretrained(self.run_params.model)
        tokenizer.pad_token = tokenizer.eos_token 
        split = f'{split}[:{self.run_params.batch_size if self.run_params.debug else f"{self.run_params.data_set_percentage}%"}]'
        ds = load_dataset('text', data_files=self.run_params.data_files, split=split)
        ds = ds.map(self.tokenize, batched=True, num_proc=4, remove_columns=['text'], fn_kwargs={'tokenizer':tokenizer})
        ds = ds.map(
            self.group_texts,
            batched=True,
            batch_size=self.block_size,
            num_proc=self.run_params.num_workers
        )
        ds.set_format(type='torch', columns=['input_ids', 'attention_mask'])
        return ds

    # Tokenize a sequence
    def tokenize(self, x, tokenizer=None):
        tokens = tokenizer(x['text'])
        return tokens
    
    def group_texts(self, examples):
        import pdb; pdb.set_trace()
        # Concatenate all texts.
        concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
        total_length = len(concatenated_examples[list(examples.keys())[0]])
        # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
            # customize this part to your needs.
        total_length = (total_length // self.block_size) * self.block_size
        if total_length == 0:
            total_length = self.block_size
        # Split by chunks of max_len.
        result = {
            k: [t[i : i + self.block_size] for i in range(0, total_length, self.block_size)]
            for k, t in concatenated_examples.items()
        }
        result["labels"] = result["input_ids"].copy()
        return result
    

# Testing

In [13]:
# Creation
from datasets import Dataset
run_params = RunParams()
data_manager = DataManager(run_params)
train_ds, valid_ds = data_manager.prepare_data()
assert isinstance(train_ds, Dataset)
import pdb; pdb.set_trace()
assert isinstance(valid_ds, Dataset)

Using custom data configuration default-04bff418a63932f2
Reusing dataset text (/root/.cache/huggingface/datasets/text/default-04bff418a63932f2/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5)






> <ipython-input-12-f201a9877dc5>(34)group_texts()
     32         import pdb; pdb.set_trace()
     33         # Concatenate all texts.
---> 34         concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
     35         total_length = len(concatenated_examples[list(examples.keys())[0]])
     36         # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can



ipdb>  n


> <ipython-input-12-f201a9877dc5>(35)group_texts()
     33         # Concatenate all texts.
     34         concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
---> 35         total_length = len(concatenated_examples[list(examples.keys())[0]])
     36         # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
     37             # customize this part to your needs.



ipdb>  n


...

--Return--
{'attention_mask': [[1, 1, 1, 1, 1, 1, ...]], 'input_ids': [[796, 569, 18354, 7496, 17740, 6711, ...]], 'labels': [[796, 569, 18354, 7496, 17740, 6711, ...]]}
> <ipython-input-12-f201a9877dc5>(47)group_texts()
     44             for k, t in concatenated_examples.items()
     45         }
     46         result["labels"] = result["input_ids"].copy()
---> 47         return result
     48 



ipdb>  n


> /usr/local/lib/python3.8/dist-packages/datasets/arrow_dataset.py(1378)does_function_return_dict()
   1376             """ Does the function returns a dict. """
   1377             fn_args = [inputs] if input_columns is None else [inputs[col] for col in input_columns]
-> 1378             processed_inputs = (
   1379                 function(*fn_args, indices, **fn_kwargs) if with_indices else function(*fn_args, **fn_kwargs)
   1380             )



ipdb>  c


IndexError: index out of bounds

In [ ]:
train_ds[1]

/usr/local/lib/python3.8/dist-packages/datasets/arrow_dataset.py:851: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.tensor(x, **format_kwargs)


{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]),
 'input_ids': tensor([  796,   569, 18354,  7496, 17740,  6711,   796,   220, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256])}